In [1]:
# test load with CIFAR10 as it has less classes similar to FACET

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from models_package import StudentModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the student model architecture and weights
student_model = torch.load('student_model.pth')
student_model.load_state_dict(torch.load('student_model_weights.pth'))
student_model = student_model.to(device)


# Modify the student model for the CIFAR-10 dataset
class ModifiedStudentModel(nn.Module):
    def __init__(self, student_model, num_classes):
        super(ModifiedStudentModel, self).__init__()
        self.features = nn.Sequential(*list(student_model.children())[:-1]) # Preserve the features part of the student model
        self.fc1 = nn.Linear(16 * 15 * 15, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 16 * 15 * 15)
        x = self.fc1(x)
        return x

modified_student_model = ModifiedStudentModel(student_model, num_classes=10)  # CIFAR-10 has 10 classes
modified_student_model = modified_student_model.to(device)

# Load the CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

# Evaluate the model on the CIFAR-10 dataset
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(modified_student_model.parameters(), lr=0.001, momentum=0.9)

correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = modified_student_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the CIFAR-10 dataset: {100 * correct / total}%')



Files already downloaded and verified
Accuracy of the network on the CIFAR-10 dataset: 13.73%
